In [1]:
import pandas as pd

# custom
# when imported sets a bunch of things in your spacy pipeline
import get_govorg_list
import govorg_matcher

# spacy
import spacy
from spacy_lookup import Entity

# Import the English language class
from spacy.lang.en import English

# make use of widescreen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# trial with a manageable number of GOV.UK pages
content_n = 1000

In [2]:
# available from GOV.UK data scientists
# data has been pre-processed for taxonomy work
# this loses us useful information such as capital letters
# will want to adjust pre-processing pipeline
df = pd.read_csv("data/11-02-19/labelled.csv",
                 usecols=["base_path", "content_id",
                          "description", "locale",
                          "title", "body", "combined_text"],
                nrows = content_n)

In [3]:
df.shape

(1000, 7)

In [4]:
govorg_matcher.text_gov_org_match(df.at[3,"body"])

['The Charity Commission']

In [5]:
# https://stackoverflow.com/questions/31674557/how-to-append-rows-in-a-pandas-dataframe-in-a-for-loop
# Get's ValueError due to overlap of NORP and GOV_ORG
# Let this be temp workaround
cols = ['base_path','content_id', 'gov_org']
lst = []
for index, row in df.iterrows():
    try:
        lst.append([row['base_path'], row['content_id'], govorg_matcher.text_gov_org_match(df.at[index,"combined_text"])])
    except ValueError:
        print('Custom Error due to Entity clash:')
        print('Row Data:',row)
        continue

df1 = pd.DataFrame(lst, columns=cols)
df1.shape

Custom Error due to Entity clash:
Row Data: base_path        /government/statistics/charitable-giving-indic...
content_id                    5dc81f86-7631-11e4-a3cb-005056011aef
description      this report contains the total amount of chari...
locale                                                          en
title                         charitable giving indicators 2011/12
body             this report contains the total amount of chari...
combined_text    charitable giving indicators 2011/12 this repo...
Name: 303, dtype: object
Custom Error due to Entity clash:
Row Data: base_path        /government/publications/dcms-bodies-settlemen...
content_id                    5e38306e-7631-11e4-a3cb-005056011aef
description      letters explaining the funding that the depart...
locale                                                          en
title            dcms bodies' settlement letters: spending revi...
body             find out more about the spending review . arts...
combined_text   

(998, 3)

This list of cited organisations is probably appropriate for writing back to the MongoDB, as JSON format can handle nested list.

In [6]:
# https://stackoverflow.com/questions/27263805/pandas-when-cell-contents-are-lists-create-a-row-for-each-element-in-the-list
# Actually we want it like this, as we need a unique id 
s = df1.apply(lambda x: pd.Series(x['gov_org']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'gov_org_name'

df1 = df1.drop('gov_org', axis=1).join(s)

df1.shape

(1150, 3)

In [7]:
# drop empty edges
df1.dropna(inplace=True)

# need id for gov_org
# call api, get dictionary to lookup and add id to it

In [8]:
govorg_dict = get_govorg_list.get_orgid_dict()

In [9]:
# create new col by looking up org_id from dict
df1['org_id'] = df1['gov_org_name'].map(govorg_dict)


df1.shape

(481, 4)

In [19]:
df1.head()

,base_path,content_id,gov_org_name,org_id
1,/government/news/charity-commission-names-furt...,5fa49c52-7631-11e4-a3cb-005056011aef,The Charity Commission,D98
2,/government/publications/trust-and-confidence-...,d0341424-12a1-4b4c-9045-2e74ba17f2d5,The Charity Commission,D98
3,/government/speeches/william-shawcross-speech-...,9245dfca-4210-41d9-9ffd-7fcc35dc1642,The Charity Commission,D98
5,/government/news/britain-honours-its-holocaust...,5b12e7a3-3db7-4710-862f-0d54ec6117b6,Foreign & Commonwealth Office,D13
6,/government/publications/esf-funding-allocated...,5f5167fc-7631-11e4-a3cb-005056011aef,Department for Work and Pensions,D10


In [10]:
# write edge list to csv
df1.to_csv('data/ContentOrgRel.csv', index=False)


In [11]:
# we'll create nodes for connected entities only, for demo purposes
# get loc of unique rows, get base_path and content_id
# create node list content_id, base_path
data = {'base_path':df1.base_path.unique(),
        'content_id':df1.content_id.unique()}
Content = pd.DataFrame(data)
print(Content.shape)
# write csv
Content.to_csv('data/Content.csv', index=False)

In [12]:
# create node list org_id, gov_org_name
data = {'gov_org_name':df1.gov_org_name.unique(),
        'org_id':df1.org_id.unique(),
       }
Org = pd.DataFrame(data)
print(Org.shape)
# write csv
Org.to_csv('data/Org.csv', index=False)

Now let's read these into Neo4j, run from Docker or use Desktop version.

Remember this is a proof of principle. We should add constraints and think more carefully about the names of things. We can adjust this as required.

We can check our files are OK by testing with: 

// check first 5 line-sample with header-mapping
LOAD CSV WITH HEADERS FROM "file:///Content.csv" AS line WITH line
RETURN line
LIMIT 5;  

To actually load the data into neo4j, we need Cypher code of something like:

MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r;  

LOAD CSV WITH HEADERS FROM "file:///Content.csv" AS csvLine CREATE (p:Content { id: csvLine.content_id, name: csvLine.base_path});  

LOAD CSV WITH HEADERS FROM "file:///Org.csv" AS csvLine CREATE (p:Org { id: csvLine.org_id, name: csvLine.gov_org_name});  


CREATE INDEX ON :Content(id);  
CREATE INDEX ON :Org(id);  

Verify these, using:  

:schema  


LOAD CSV WITH HEADERS FROM "file:///ContentOrgRel.csv" AS csvLine MATCH (o1:Content { id: tostring(csvLine.content_id)}), (o2:Org { id: tostring(csvLine.org_id)}) CREATE (o1)-[:REFERs_TO]->(o2);


